<font size="5" color='red'> <b> This script does not include the correction for the machine<b> </font>

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import linregress
from scipy.signal import butter,filtfilt, find_peaks

In [ ]:
###### CELL TO DO CHANGES

## For my experiments I axially compressed the sample in a hoek cell. Each cycle was conducted at different confining pressures from 10 to 50 MPa

# Import data
df_exp_PB = pd.read_csv('File.csv')                                                            # Read the file

# Indexes of each cycle range from the file that the scrip will read and plot (see example)
n = 0
m = 0


# Limits for the linear regression
lim1 = 0
lim2 = 0


# It is the offset in the slope equation  y=ax+b to change the position of your line. Normally no more than 0.5
disp = 0.1

In [ ]:
df_exp_PB_1 = df_exp_PB.iloc[n:m]

stress1 = df_exp_PB_1['Stress [MPa]']             # Apply filter to stress
strain1 = df_exp_PB_1['Strain Ax [-]']            # Apply filter to strain

# print(stress1)

stress2 = stress1.to_numpy()
strain2 = strain1.to_numpy()


In [ ]:
###ONLY FOR LINEAR REGRESSION
# Plot the young modulus slope
stress_range = stress1[(stress1 >= lim1) & (stress1 <= lim2)]                               # stress filtered for values more than 15 less 33
strain_range = strain1[(stress1 >= lim1) & (stress1 <= lim2)]                               # strain filtered for values more than 15 less 33


## Linear regression to fit the linear slope corresponding to the Young's
linear_regression_output = linregress(strain_range, stress_range)
print(linear_regression_output)
E = linear_regression_output[0]
b = linear_regression_output[1]
ElasticM = f'The elastic modulus is {round(E/1000,1)} GPa'

stress_offset = E*(strain1) + b - disp                                                                # y=mx+b


## Find yield strength same as with UCS but taking the chosen indexes into account
for i in range(n, m):
    if stress_offset[i] >= stress1[i]:
        sx1 = strain1[i-1]
        sy1 = stress1[i-1]
        sy2 = stress1[i]
        sx2 = strain1[i]

        ox1 = strain1[i-1]
        oy1 = stress_offset[i-1]
        ox2 = stress1[i]
        oy2 = stress_offset[i]
        break

# print(sx1, sy1, sx2, sy2, ox1, oy1, ox2, oy2)

x1 = ox1
y1 = oy1
x2 = ox2
y2 = oy2

x3 = sx1
y3 = sy1
x4 = sx2
y4 = sy2

YS = ( (x1*y2 - y1*x2)*(y3-y4) - (y1-y2)*(x3*y4-y3*x4) ) / ( ( (x1-x2)*(y3-y4))- ( (y1-y2)*(x3-x4)))
print(YS)

xi = (YS-b) / (E)
yi = YS

print('(xi,yi)',xi,yi)

In [ ]:
# Maximum strength
y = stress2.max()                                                                               # Find maximum stress
max_index = stress2.argmax()
# print(max_index)
x = strain2[max_index]


## Fracture strength: point of strain where the material physically separates. 
# At this point, the strain reaches its maximum value and the material actually fractures, even though the corresponding stress may be less than the ultimate strength at this point

fract_strain =  strain2.max()
max_index_fract = strain2.argmax()
fract_stress = stress2[max_index_fract]
# fract_stress = np.interp(fract_strain, strain1, stress1)
FractS = f'The fracture strength is {round(fract_stress,1)} MPa'
print(fract_strain, fract_stress)

In [ ]:
### INTERACTIVE PLOTTING
## Plot all the data. This plots all the cycles together to visualize the data
ax = px.line(df_exp_PB, x = 'Strain Ax [-]', y = 'Stress [MPa]', hover_data=['Index'], width=700, height=700, title='PB20_TRI - Full test')

ax.show()

ax.write_html("Folder/name.html")                       # It saves the plot as an interactive plot. You can also save the plot from the direct output

In [ ]:
### Plot loops. Thi part of the scrip plots each cycle individually
fig = go.Figure()
fig.add_trace(go.Scatter(x=strain1, y=stress1))

fig.update_layout(showlegend=False, autosize=False, width=700, height=700, 
    title="PB20 LOOPs",
    xaxis_title="Strain Ax [-]",
    yaxis_title="Stress [MPa]",
    font=dict(
        family="New Times Roman",
        size=15,
        color="RebeccaPurple"))

fig.show()

ax.write_html("Folder/name.html")                       # It saves the plot as an interactive plot. You can also save the plot from the direct output

In [ ]:
### All info
fig = go.Figure()
fig.add_trace(go.Scatter(x=strain1, y=stress1))
fig.add_trace(go.Scatter(x=strain1, y=stress_offset))

## Max strength
fig.add_annotation(x=x, y=y,
            text="Strain={:.4f}, Stress={:.4f}".format(x, y),
            showarrow=True,
            xanchor="right",
            arrowhead=1)

## Elastic Moduli text
fig.add_annotation(x=x, y=20,
            text=ElasticM,
            showarrow=False,
            arrowhead=1)

## Fracture strength text
fig.add_annotation(x=x, y=15,
            text=FractS,
            showarrow=False,
            arrowhead=1)

## Yield point text
fig.update_layout(showlegend=False, autosize=False, width=700, height=700, 
    title="All info",
    xaxis_title="Strain Ax [-]",
    yaxis_title="Stress [MPa]",
    font=dict(
        family="New Times Roman",
        size=15,
        color="RebeccaPurple"))


fig.add_annotation(x=xi, y=yi,
            text="Strain={:.4f}, Stress={:.4f}".format(x1, y1),
            xanchor="right",
            showarrow=True,
            arrowhead=1)

fig.update_traces(textposition='top center')
fig.show()

ax.write_html("Folder/name.html")                       # It saves the plot as an interactive plot. You can also save the plot from the direct output